In [3]:
import os
import pandas as pd
import gradio as gr
from pydantic import BaseModel, Field

import langchain
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

Enable Langsmith

In [11]:
from langchain.callbacks.manager import tracing_v2_enabled
from langsmith import Client
client = Client()

LANGCHAIN_TRACING_V2=os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT="palm-ragfusion-shadow-99"  # if not specified, defaults to "default"
LANGCHAIN_ENDPOINT=os.getenv("LANGCHAIN_ENDPOINT")
OPENAI_API_KEY=OPENAI_API_KEY

In [4]:
langchain.debug = False
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

In [5]:
data_dir_path = os.path.join(os.getcwd(), 'data')
NUM_ROWS_TO_RETURN = 5

table_1_df = pd.read_csv(os.path.join(data_dir_path, 'legal_entries_a.csv'))
table_2_df = pd.read_csv(os.path.join(data_dir_path, 'legal_entries_b.csv'))
template_df = pd.read_csv(os.path.join(data_dir_path, 'legal_template.csv'))

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [8]:
transform_model = GoogleGenerativeAI(model="gemini-pro",temperature=0, google_api_key=os.getenv("GOOGLE_API_KEY"))

natural_language_model = GoogleGenerativeAI(model="gemini-pro",temperature=0.1, google_api_key=os.getenv("GOOGLE_API_KEY"))

In [9]:
# TODO: add validation to models, coupled with retry mechanism in chain
class TableMappingEntry(BaseModel):
    '''A single row in a table mapping. Describes how a single column in a source table maps to a single column in a target table, including any necessary transformations, and their explanations.'''
    source_column_name: str = Field(..., description="Name of the column in the source table.")
    target_column_name: str = Field(..., description="Name of the column in the target table, to which the source column maps.")
    value_transformations: str = Field(..., description="Transformations needed make the source values match the target values. If unncecessary, write 'NO_TRANSFORM'.")
    explanation: str = Field(..., description="One-sentence explanation of this row (source-target mapping/transformation). Include any information that might be relevant to a software engineer building an ETL pipeline with this document.")

class TableMapping(BaseModel):
    '''A list of table mappings collectively describe how a source table should be transformed to match the schema of a target table.'''
    table_mappings: list[TableMappingEntry] = Field(..., description="A list of table mappings.")
    
analyst_prompt_str = '''
You are a Data Scientist, who specializes in generating schema mappings for use by Software Engineers in ETL pipelines.

Head of `source_csv`:

{source_1_csv_str}

Head of `target_csv`:

{target_csv_str}

Your job is to generate a thorough, precise summary of how `source_csv` should be transformed to adhere exactly to the `target_csv` schema.

For each column in the `source_csv`, you must communicate which column in the `target_csv` it maps to, and how the values in the `source_csv` column should be transformed to match those in the `target_csv`.
You can assume the rows are aligned: that is, the first row in `source_csv` corresponds to the first row in `target_csv`, and so on.

Remember:
1. Which column in `target_csv` it maps to. You should consider the semantic meaning of the columns, not just the character similarity. 

Example mappings:
- 'MunICipality' in `source_csv` should map to 'City' in `target_csv`.
- 'fullname' in `source_csv` should map to both 'FirstName' and 'LastName' in `target_csv`. You must explain this transformation, as well, including the target sequencing of first and last name.

Example transformations:
- If date in `source_csv` is `2020-01-01` and date in `target_csv` is `01/01/2020`, explain exactly how this should be transformed and the reasoning behind it.
- If city in `source_csv` is `New York` and city in `target_csv` is `NEW YORK` or `NYC`, explain exactly how this should be transformed and the reasoning behind it.

Lastly, point out any other oddities, such as duplicate columns, erroneous columns, etc.

{format_instructions}

Remember:
- Be concise: you are speaking to engineers, not customers.
- Be precise: all of these values are case sensitive. Consider casing for city names, exact prefixes for identifiers, ordering of people's names, etc.
- DO NOT include commas, quotes, or any other characters that might interfere with JSON serialization or CSV generation

Your response:
'''

def get_data_str_from_df_for_prompt(df, use_head=True, num_rows_to_return=NUM_ROWS_TO_RETURN):
    data = df.head(num_rows_to_return) if use_head else df.tail(num_rows_to_return)
    return f'<df>\n{data.to_markdown()}\n</df>'

table_mapping_parser = PydanticOutputParser(pydantic_object=TableMapping)
analyst_prompt = ChatPromptTemplate.from_template(
    template=analyst_prompt_str, 
    partial_variables={'format_instructions': table_mapping_parser.get_format_instructions()},
)

mapping_chain = analyst_prompt | transform_model | table_mapping_parser
table_mapping: TableMapping = mapping_chain.invoke({"source_1_csv_str": get_data_str_from_df_for_prompt(table_1_df), "target_csv_str": get_data_str_from_df_for_prompt(template_df)})

In [13]:
SPEC_WRITER_PROMPT_STR = '''
You are an expert product manager and technical writer for a software company, who generates clean, concise, precise specification documents for your employees.
Your job is to write a plaintext spec for a python script for a software engineer to develop a component within an ETL pipeline.

This document must include 100% of the information your employee needs to write a successful script to transform source_df to target_df.
However, DO NOT include the original table_mapping. Your job is to translate everything into natural language.

Here is a stringified Pandas DataFrame that describes the mapping and the transformation steps:

{table_mapping}

You must translate this into clean, concise, and complete instructions for your employee.

This document should be formatted like a technical document in plaintext. Do not include code or data.

This document must include:
- Overview
- Input (source_df)
- Output (target_df)
- Exact column mapping
- Exact transformation steps for each column
- Precise instructions for what this script should do
- Do not modify the source_df. Create a new dataframe named target_df.
- This script should never include the source data. It should only include the transormations required to create the target_df.
- Return the target_df

You will never see this employee. They cannot contact you. You will never see their code. You must include 100% of the information they need to write a successful script.
Remember:
- Clean: No extra information, no formatting aside from plaintext
- Concise: Your employees benefit from brevity
- Precise: your words must be unambiguous, exact, and full represent a perfect translation of the pandas dataframe.

Your response:
'''
spec_writer_prompt = ChatPromptTemplate.from_template(SPEC_WRITER_PROMPT_STR)

spec_writer_chain = spec_writer_prompt | natural_language_model | StrOutputParser()
spec_str = spec_writer_chain.invoke({"table_mapping": str(table_mapping.model_dump()['table_mappings'])})

In [15]:
pd.DataFrame(table_mapping.model_dump()['table_mappings']).to_dict()

{'source_column_name': {0: 'case_date',
  1: 'lastname',
  2: 'firstname',
  3: 'case_type',
  4: 'case_id',
  5: 'court_fee',
  6: 'jurisdiction',
  7: 'judge_last_name'},
 'target_column_name': {0: 'CaseDate',
  1: 'FullName',
  2: 'FullName',
  3: 'CaseType',
  4: 'CaseID',
  5: 'Fee',
  6: 'Jurisdiction',
  7: 'FullName'},
 'value_transformations': {0: 'NO_TRANSFORM',
  1: "CONCATENATE(firstname, ' ', lastname)",
  2: 'NO_TRANSFORM',
  3: 'NO_TRANSFORM',
  4: "REPLACE(case_id, 'CR-', 'CASE-')",
  5: 'NO_TRANSFORM',
  6: 'CAPITALIZE(jurisdiction)',
  7: 'NO_TRANSFORM'},
 'explanation': {0: 'The date format is the same in both tables.',
  1: 'The target table has a single column for the full name, so we concatenate the first and last names from the source table.',
  2: 'The first name is already included in the concatenation in the previous row.',
  3: 'The case types are the same in both tables.',
  4: "The target table uses the prefix 'CASE-' for case IDs, so we replace 'CR-' with 

In [16]:
ENGINEER_PROMPT_STR = '''
You are a Senior Software Engineer, who specializes in writing Python code for ETL pipelines.
Your Product Manager has written a spec for a new transormation script. You must follow this document exactly, write python code that implements the spec, validate that code, and then return it.
Your output should only be python code in Markdown format, eg:
    ```python
    ....
    ```"""
Do not return any additional text / explanation. This code will be executed by a robot without human intervention.

Here is the technical specification for your code:

{spec_str}

Remember: return only clean python code in markdown format. The python interpreter running this code will already have `source_df` as a local variable.

Your must return `target_df` at the end.
'''
engineer_prompt = ChatPromptTemplate.from_template(ENGINEER_PROMPT_STR)

# engineer_chain = engineer_prompt | transform_model | StrOutputParser() | PythonAstREPLTool(locals={'source_df': table_1_df}).run
# table_1_df_transformed = engineer_chain.invoke({"spec_str": spec_str})
engineer_chain = engineer_prompt | transform_model | StrOutputParser()
transform_code = engineer_chain.invoke({"spec_str": spec_str})

In [19]:
def generate_mapping_code(table_mapping_df) -> str:
    writer_prompt = ChatPromptTemplate.from_template(SPEC_WRITER_PROMPT_STR)
    engineer_prompt = ChatPromptTemplate.from_template(ENGINEER_PROMPT_STR)
    
    writer_chain = writer_prompt | transform_model | StrOutputParser()
    engineer_chain = {"spec_str": writer_chain} | engineer_prompt | transform_model | StrOutputParser()
    return engineer_chain.invoke({"table_mapping": str(table_mapping_df)})

In [20]:
generate_mapping_code()

TypeError: generate_mapping_code() missing 1 required positional argument: 'table_mapping_df'

In [21]:
def show_mapping(file):
    # TODO: add code
    return pd.DataFrame(table_mapping.dict()['table_mappings'])
demo = gr.Interface(fn=show_mapping, inputs=["file"], outputs='dataframe')
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [109]:
def _sanitize_python_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

def show_code(button):
    # TODO: add code
    return _sanitize_python_output(transform_code)
check_mapping_text = 'How does that mapping look? \n\nFeel free to update it: your changes will be incorporated! \n\nWhen you are ready, click the Submit below, and the mapping code will be generated for your approval.'
demo = gr.Interface(fn=show_code, inputs=[gr.Textbox(value=check_mapping_text, interactive=False)], outputs=[gr.Code(language="python")])
demo.launch()

Running on local URL:  http://127.0.0.1:7939

To create a public link, set `share=True` in `launch()`.


In [110]:
def get_transformed_table(button):
    return template_df, PythonAstREPLTool(locals={'source_df': table_1_df}).run(transform_code)
check_mapping_text = 'How does that code look? \n\nWhen you are ready, click the Submit button and the transformed source file will be transformed.'
demo = gr.Interface(
    fn=get_transformed_table,
    inputs=[gr.Textbox(value=check_mapping_text, interactive=False)],
    outputs=[gr.Dataframe(label='Template Table (target)'), gr.Dataframe(label='Table 1 (transformed)')],
    layout="column",
    examples=[[1]],
)
demo.launch()

/var/folders/lx/3ksh07r96gn2v7b8mb__3mpc0000gn/T/ipykernel_94012/4236222443.py:4: GradioDeprecationWarning: `layout` parameter is deprecated, and it has no effect
  demo = gr.Interface(


Running on local URL:  http://127.0.0.1:7940

To create a public link, set `share=True` in `launch()`.


In [43]:
def _sanitize_python_output(text: str):
    _, after = text.split("```python")
    return after.split("```")[0]

import io
def generate_code(val):
    return '# check this out'

def export_csv(d):
    filepath = "output.csv"
    d.to_csv(filepath)
    return gr.File.update(value=filepath, visible=True)

def get_table_mapping(source_df, template_df):
    # table_mapping_df = pd.DataFrame(table_mapping.dict()['table_mappings'])
    return pd.DataFrame({'a': [1,2,3], 'b': [4,5,6]})

def process_csv_text(temp_file):
    if isinstance(temp_file, str):
      df = pd.read_csv(io.StringIO(temp_file))
    else:
      df = pd.read_csv(temp_file.name)
    return df

def generate_step_markdown(step_number: int, subtitle: str):
    return gr.Markdown(f"# Step {step_number}\n\n ### {subtitle}")

# TODO: parameterize
def export_table_mapping(d):
    filename = "source_template_mapping.csv"
    d.to_csv(filename)
    return gr.File.update(value=filename, visible=True)

def export_python_code(val):
    filename = "transformation_code.py"
    with open(filename, "w") as f:
        f.write(val)
    return gr.File.update(value=filename, visible=True)

def export_transformed_source(d):
    filename = "transformed_source.csv"
    d.to_csv(filename)
    return gr.File.update(value=filename, visible=True)

with gr.Blocks() as demo:
    # STEP 1
    generate_step_markdown(1, "Upload a Template CSV (target schema) and a Source CSV.")
    with gr.Row():
        with gr.Column():
            upload_template_btn = gr.UploadButton(label="Upload Template File", file_types = ['.csv'], live=True, file_count = "single")
            template_df = gr.Dataframe(type="pandas")
            upload_template_btn.upload(fn=process_csv_text, inputs=upload_template_btn, outputs=template_df)
        with gr.Column():
            upload_source_button = gr.UploadButton(label="Upload Source File", file_types = ['.csv'], live=True, file_count = "single")
            source_df = gr.Dataframe(type="pandas")
            upload_source_button.upload(fn=process_csv_text, inputs=upload_source_button, outputs=source_df)
    
    # STEP 2
    generate_step_markdown(2, "Generate mapping from Source to Template. Once generated, you can edit the values directly in the table below.")
    with gr.Row():
        generate_mapping_btn = gr.Button(value="Generate Mapping", variant="primary")
    with gr.Row():
        table_mapping_df = gr.DataFrame(type="pandas")
        generate_mapping_btn.click(fn=get_table_mapping, inputs=[source_df, template_df], outputs=[table_mapping_df])
    
    with gr.Row():
        save_mapping_btn = gr.Button(value="Save Mapping", variant="secondary")
    with gr.Row():
        csv = gr.File(interactive=False, visible=False)
        save_mapping_btn.click(export_table_mapping, table_mapping_df, csv)
        mapping_file = gr.File(label="Downloaded File", visible=False)
        mapping_file.change(lambda x: x, mapping_file, table_mapping_df)
    
    # STEP 3
    generate_step_markdown(3, "Generate python code to transform Source to Template, using the generated mapping.")
    with gr.Row():
        generate_code_btn = gr.Button(value="Generate Code from Mapping", variant="primary")
    with gr.Row():
        code_block = gr.Code(language="python")
        generate_code_btn.click(fn=generate_code, outputs=[code_block])

    with gr.Row():
        save_code_btn = gr.Button(value="Save Code", variant="secondary")
    with gr.Row():
        text = gr.File(interactive=False, visible=False)
        save_code_btn.click(export_python_code, code_block, text)
        code_file = gr.File(label="Downloaded File", visible=False)
        code_file.change(lambda x: x, code_file, code_block)

    # STEP 4
    generate_step_markdown(4, "Transform the Source CSV into the Template CSV using the generated code.")
    with gr.Row():
        transform_btn = gr.Button(value="Transform Source", variant="primary")
    with gr.Row():
        source_df_transformed = gr.Dataframe(type="pandas")
        transform_btn.click(lambda x: x, inputs=[source_df], outputs=[source_df_transformed])

    with gr.Row():
        save_transformed_source_btn = gr.Button(value="Save Transformed Source", variant="secondary")
    with gr.Row():
        csv = gr.File(interactive=False, visible=False)
        save_transformed_source_btn.click(export_transformed_source, source_df_transformed, csv)
        transform_file = gr.File(label="Downloaded File", visible=False)
        transform_file.change(lambda x: x, transform_file, source_df_transformed)

    
    
    # with gr.Row():
    #     with gr.Column():
    #         gr.Dataframe(label='Target (template)', type='pandas', value=template_df)
    #     with gr.Column():
    #         gr.Dataframe(label='Source (transformed)', type='pandas', value=PythonAstREPLTool(locals={'source_df': table_1_df}).run(transform_code))

            
            



# def mock_ocr(f):
#     return [[1, 2, 3], [4, 5, 6]]



# with gr.Blocks() as demo:
#     with gr.Row():
#         file = gr.File(label="PDF file", file_types=[".pdf"])
#         dataframe = gr.Dataframe()
        
#         with gr.Column():
#             button = gr.Button("Export")
#             csv = gr.File(interactive=False, visible=False)
            
            
#     file.change(mock_ocr, file, dataframe)
#     button.click(export_csv, dataframe, csv)
        
# demo.launch()




    # with gr.Column():
    #     gr.Markdown("## Mapping from Source to Template")
    #     with gr.Row():
    #         table_mapping_df = pd.DataFrame(table_mapping.dict()['table_mappings'])
    #         gr.DataFrame(value=table_mapping_df)
    #         save_mapping_btn = gr.Button(value="Save Mapping", variant="secondary")
    #         save_mapping_btn.click(fn=lambda : save_csv_file(table_mapping_df, 'table_mapping'))

    # with gr.Row():
    #     test = gr.Markdown()
    #     generate_code_btn = gr.Button(value="Generate Code from Mapping", variant="primary")
    #     generate_code_btn.click(fn=generate_code, outputs=test)

    # with gr.Column():
    #     gr.Markdown("## Here is the code that will be used to transform the source file into the template schema:")
    #     gr.Code(language="python", value=_sanitize_python_output(transform_code))

    # with gr.Row():
    #     gr.Button(value="Transform Source", variant="primary", trigger="transform_source")
    #     gr.Button(value="Save Code", variant="secondary", trigger="save_code")
    
    # with gr.Row():
    #     with gr.Column():
    #         gr.Dataframe(label='Target (template)', type='pandas', value=template_df)
    #     with gr.Column():
    #         gr.Dataframe(label='Source (transformed)', type='pandas', value=PythonAstREPLTool(locals={'source_df': table_1_df}).run(transform_code))

demo.launch()

/var/folders/lx/3ksh07r96gn2v7b8mb__3mpc0000gn/T/ipykernel_13584/3295794268.py:55: GradioUnusedKwargWarning: You have unused kwarg parameters in UploadButton, please remove them: {'live': True}
  upload_template_btn = gr.UploadButton(label="Upload Template File", file_types = ['.csv'], live=True, file_count = "single")
/var/folders/lx/3ksh07r96gn2v7b8mb__3mpc0000gn/T/ipykernel_13584/3295794268.py:59: GradioUnusedKwargWarning: You have unused kwarg parameters in UploadButton, please remove them: {'live': True}
  upload_source_button = gr.UploadButton(label="Upload Source File", file_types = ['.csv'], live=True, file_count = "single")
/Users/andybryant/Desktop/projects/zero-mapper/venv/lib/python3.9/site-packages/gradio/utils.py:841: UserWarning: Expected 1 arguments for function <function generate_code at 0x12c23b820>, received 0.
  warnings.warn(
/Users/andybryant/Desktop/projects/zero-mapper/venv/lib/python3.9/site-packages/gradio/utils.py:845: UserWarning: Expected at least 1 argumen

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [176]:
source_df

dataframe
